<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W4/assignment/C1W4_Assignment_traducido.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Semana 4: Manejo de imágenes complejas - Conjunto de datos feliz o triste

En esta tarea se utilizará el conjunto de datos Feliz o Triste, que contiene 80 imágenes de caras tipo emoji, 40 felices y 40 tristes.

Cree una red neuronal convolucional que se entrene con una precisión del 99,9% en estas imágenes, y que cancele el entrenamiento al alcanzar este umbral de precisión.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

# Week 4: Handling Complex Images - Happy or Sad Dataset

In this assignment you will be using the happy or sad dataset, which contains 80 images of emoji-like faces, 40 happy and 40 sad.

Create a convolutional neural network that trains to 99.9% accuracy on these images,  which cancels training upon hitting this training accuracy threshold.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

## Cargar y explorar los datos

Comience por echar un vistazo a algunas imágenes del conjunto de datos.

Observa que todas las imágenes están contenidas en el directorio `./data/`. 

Este directorio contiene dos subdirectorios `feliz/` y `triste/` y cada imagen se guarda en el subdirectorio relacionado con la clase a la que pertenece.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Load and explore the data

Begin by taking a look at some images of the dataset.

Notice that all the images are contained within the `./data/` directory. 

This directory contains two subdirectories `happy/` and `sad/` and each image is saved under the subdirectory related to the class it belongs to.

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

base_dir = "./data/"
happy_dir = os.path.join(base_dir, "happy/")
sad_dir = os.path.join(base_dir, "sad/")

print("Sample happy image:")
plt.imshow(load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}"))
plt.show()

print("\nSample sad image:")
plt.imshow(load_img(f"{os.path.join(sad_dir, os.listdir(sad_dir)[0])}"))
plt.show()


Es genial poder ver ejemplos de las imágenes para entender mejor el espacio-problema con el que se está tratando. 

Sin embargo, todavía falta alguna información relevante como la resolución de la imagen (aunque matplotlib renderiza las imágenes en una cuadrícula proporcionando una buena idea de estos valores) y el valor máximo de los píxeles (esto es importante para normalizar estos valores). Para ello se puede utilizar Keras como se muestra en la siguiente celda:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

It is cool to be able to see examples of the images to better understand the problem-space you are dealing with. 

However there is still some relevant information that is missing such as the resolution of the image (although matplotlib renders the images in a grid providing a good idea of these values) and the maximum pixel value (this is important for normalizing these values). For this you can use Keras as shown in the next cell:

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

# Cargar el primer ejemplo de una cara feliz
sample_image  = load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}")

# Convierte la imagen en su representación de matriz numpy
sample_array = img_to_array(sample_image)

print(f"Each image has shape: {sample_array.shape}")

print(f"The maximum pixel value used is: {np.max(sample_array)}")

Parece que las imágenes tienen una resolución de 150x150. **Esto es muy importante porque este será el tamaño de entrada de la primera capa de tu red.** 

**La última dimensión se refiere a cada uno de los 3 canales RGB que se utilizan para representar las imágenes en color.**

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Looks like the images have a resolution of 150x150. **This is very important because this will be the input size of the first layer in your network.** 

**The last dimension refers to each one of the 3 RGB channels that are used to represent colored images.**

## Definición de la llamada de retorno

Como ya has codificado la llamada de retorno responsable de detener el entrenamiento (una vez que se alcanza un nivel de precisión deseado) en las dos tareas anteriores, esta vez ya se proporciona para que puedas centrarte en los otros pasos:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Defining the callback

Since you already have coded the callback responsible for stopping training (once a desired level of accuracy is reached) in the previous two assignments this time it is already provided so you can focus on the other steps:

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

Una nota rápida sobre las devoluciones de llamada: 

Hasta ahora sólo has utilizado la llamada de retorno `on_epoch_end` pero hay muchas más. Por ejemplo, puede que quieras echar un vistazo a la devolución de llamada [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping), que te permite guardar los mejores pesos para tu modelo.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

A quick note on callbacks: 

So far you have used only the `on_epoch_end` callback but there are many more. For example you might want to check out the [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) callback, which allows you to save the best weights for your model.

## Pre-procesamiento de los datos

Keras proporciona un gran soporte para el preprocesamiento de datos de imágenes. Se puede lograr mucho utilizando la clase `ImageDataGenerator`. Asegúrate de revisar los [docs](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) si te atascas en el siguiente ejercicio. En particular, debes prestar atención al argumento `rescale` al instanciar el `ImageDataGenerator` y al método [`flow_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory).

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Pre-processing the data

Keras provides great support for preprocessing image data. A lot can be accomplished by using the `ImageDataGenerator` class. Be sure to check out the [docs](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) if you get stuck in the next exercise. In particular you might want to pay attention to the `rescale` argument when instantiating the `ImageDataGenerator` and to the [`flow_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory) method.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# GRADED FUNCTION: image_generator
def image_generator():
    ### START CODE HERE

    # Instanciar la clase ImageDataGenerator.
    # Recuerda establecer el argumento de reescalado.
    train_datagen = None

    # Especifica el método para cargar las imágenes desde un directorio y pasa los argumentos adecuados:
    # - directory: debe ser una ruta relativa al directorio que contiene los datos
    # - targe_size: establecerlo igual a la resolución de cada imagen (excluyendo la dimensión de color)
    # - batch_size: número de imágenes que el generador produce cuando se le pide un siguiente lote. Establézcalo en 10.
    # - class_mode: Cómo se representan las etiquetas. Debe ser uno de "binario", "categórico" o "disperso".
    # Escoge el que mejor se adapte aquí dado que las etiquetas van a ser etiquetas binarias 1D.
    train_generator = train_datagen.flow_from_directory(directory=None,
                                                        target_size=(None, None),
                                                        batch_size=None,
                                                        class_mode=None)
    ### END CODE HERE

    return train_generator
    

In [ ]:
# Guarda tu generador en una variable
gen = image_generator()

**Salida esperada:**
```
Se han encontrado 80 imágenes pertenecientes a 2 clases.
```

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

**Expected Output:**
```
Found 80 images belonging to 2 classes.
```

## Creación y entrenamiento del modelo

Finalmente, completa la función `train_happy_sad_model` que aparece a continuación. Esta función debería devolver tu red neuronal.

**Tu modelo debe alcanzar una precisión del 99,9% o más antes de 15 épocas para aprobar esta tarea.

**Pistas:**
- Puedes probar cualquier arquitectura para la red pero ten en cuenta que el modelo funcionará mejor con 3 capas convolucionales. 


- En caso de que necesites ayuda extra puedes consultar algunos consejos al final de este cuaderno.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Creating and training your model

Finally, complete the `train_happy_sad_model` function below. This function should return your  neural network.

**Your model should achieve an accuracy of 99.9% or more before 15 epochs to pass this assignment.**

**Hints:**
- You can try any architecture for the network but keep in mind that the model will work best with 3 convolutional layers. 


- In case you need extra help you can check out some tips at the end of this notebook.

In [ ]:
from tensorflow.keras import optimizers, losses

# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model(train_generator):

    # Instanciar el Callback
    callbacks = myCallback()

    ### START CODE HERE

    # Definir el modelo
    model = tf.keras.models.Sequential([
        None,
    ])

    # Compilar el modelo
    # Selecciona una función de pérdida compatible con la última capa de tu red
    model.compile(loss=losses.None,
                  optimizer=optimizers.None,
                  metrics=['accuracy']) 
    


    # Entrena el modelo
    # Tu modelo debería alcanzar la precisión deseada en menos de 15 epochs.
    # Puede codificar hasta 20 épocas en la función de abajo, pero la llamada de retorno debe activarse antes de 15.
    history = model.fit(x=None,
                        epochs=None,
                        callbacks=[None]
                       ) 
    
    ### END CODE HERE
    return history

In [ ]:
hist = train_happy_sad_model(gen)

Si ves el mensaje que se definió en el callback impreso después de menos de 15 épocas significa que tu callback funcionó como se esperaba y el entrenamiento fue exitoso. También puede comprobarlo ejecutando la siguiente celda:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

If you see the message that was defined in the callback printed out after less than 15 epochs it means your callback worked as expected and training was successful. You can also double check by running the following cell:

In [ ]:
print(f"Your model reached the desired accuracy after {len(hist.epoch)} epochs")

Si su llamada de retorno no detiene el entrenamiento, una de las causas puede ser que haya compilado su modelo utilizando una métrica diferente a la de `precisión` (como `acc`). Asegúrate de que has configurado la métrica como `precisión`. Puedes comprobarlo ejecutando la siguiente celda:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

If your callback didn't stop training, one cause might be that you compiled your model using a metric other than `accuracy` (such as `acc`). Make sure you set the metric to `accuracy`. You can check by running the following cell:

In [ ]:
if not "accuracy" in hist.model.metrics_names:
    print("Use 'accuracy' as metric when compiling your model.")
else:
    print("The metric was correctly defined.")

## ¿Necesitas más ayuda?

Ejecute la siguiente celda para ver algunos consejos adicionales para la arquitectura del modelo.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Need more help?

Run the following cell to see some extra tips for the model's architecture.

In [ ]:
import base64

encoded_answer = "ClNvbWUgaGVscGZ1bCB0aXBzIGluIGNhc2UgeW91IGFyZSBzdHVjazoKCiAgICAtIEEgZ29vZCBmaXJzdCBsYXllciB3b3VsZCBiZSBhIENvbnYyRCBsYXllciB3aXRoIGFuIGlucHV0IHNoYXBlIHRoYXQgbWF0Y2hlcyAKICAgIHRoYXQgb2YgZXZlcnkgaW1hZ2UgaW4gdGhlIHRyYWluaW5nIHNldCAoaW5jbHVkaW5nIHRoZSBjb2xvciBkaW1lbnNpb24pCiAgICAKICAgIC0gVGhlIG1vZGVsIHdpbGwgd29yayBiZXN0IHdpdGggMyBjb252b2x1dGlvbmFsIGxheWVycwogICAgCiAgICAtIFRoZXJlIHNob3VsZCBiZSBhIEZsYXR0ZW4gbGF5ZXIgaW4gYmV0d2VlbiBjb252b2x1dGlvbmFsIGFuZCBkZW5zZSBsYXllcnMKICAgIAogICAgLSBUaGUgZmluYWwgbGF5ZXIgc2hvdWxkIGJlIGEgRGVuc2UgbGF5ZXIgd2l0aCB0aGUgbnVtYmVyIG9mIHVuaXRzIGFuZCAKICAgIGFjdGl2YXRpb24gZnVuY3Rpb24gdGhhdCBzdXBwb3J0cyBiaW5hcnkgY2xhc3NpZmljYXRpb24uCg=="
encoded_answer = encoded_answer.encode('ascii')
answer = base64.b64decode(encoded_answer)
answer = answer.decode('ascii')

print(answer)

**Felicitaciones por haber terminado la última tarea de este curso.**

Has implementado con éxito una CNN para ayudarte en la tarea de clasificación de imágenes complejas. Buen trabajo.

**Sigue así.**

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

**Congratulations on finishing the last assignment of this course!**

You have successfully implemented a CNN to assist you in the classification task for complex images. Nice job!

**Keep it up!**